# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MessageCats", engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
   
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# perform train test split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train pipleline
pipeline.fit(X_train,Y_train)
print(pipeline.score(X_train,Y_train))

0.9961855355508087


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

#from sklearn.metrics import accuracy_score
#categories = Y.columns
#for i in range(len(categories)):
#    acc = accuracy_score(y_test.iloc[:,i],res[:,i])
#    print("Accuracy for {} : {}".format(categories[i],acc))

#categories = Y.columns
#for category in categories:
#    print(category)
#    print(classification_report(Y_test[category], Y_pred[category]))

categories = Y.columns
Y_pred2 = pd.DataFrame(Y_pred, columns = categories)
for category in categories:
    print(category)
    print(classification_report(Y_test[category], Y_pred2[category], zero_division=0))

related
              precision    recall  f1-score   support

           0       0.72      0.30      0.42      1514
           1       0.82      0.96      0.88      4986
           2       0.53      0.44      0.48        54

    accuracy                           0.80      6554
   macro avg       0.69      0.57      0.60      6554
weighted avg       0.79      0.80      0.77      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5409
           1       0.89      0.43      0.58      1145

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000001CA716370D8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [38]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': (True, False),
    #'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [39]:
cv.fit(X_train, Y_train)
print("\nBest Score:", cv.best_score_)
print("\nBest Parameters:", cv.best_params_)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
print('Accuracy: {}\n'.format((Y_pred == Y_test).mean().mean()))

Accuracy: 0.9469068931610892



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
X2 = pd.DataFrame(X_test)
X2['length'] = X2['message'].apply(lambda x: len(x))
X2

,message,length
23203,This was the first Boko Haram raid in a few da...,189
12258,Pr Sahil Khan of Peshawar Ap kaisi teak hi how...,119
1208,I'd like to know what will be done for police ...,133
1437,i need all the name of american superstar who ...,68
25303,The storms damaged hundreds of thatched houses...,99
...,...,...
21695,Transitional shelters are approximately 11 ft ...,115
3883,PLEASE I NEED HELP FOR MY STUDIES I WANT TO CO...,127
17089,"Colombo, July 29 -- The Food and Agriculture O...",262
9319,"Hi! please I need Information about job, I've ...",258


In [37]:
from sklearn.base import BaseEstimator, TransformerMixin

class MessageLength(BaseEstimator, TransformerMixin):
    
    def message_len(self, text):
        return len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.message_len)
        return pd.DataFrame(X_tagged)

def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('message_len', MessageLength())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])   

    return pipeline

model = model_pipeline()
model.fit(X_train, Y_train)
Y_model_pred = model.predict(X_test)
print(model.score(X_train,Y_train))
print('Accuracy: {}\n'.format((Y_model_pred == Y_test).mean().mean()))

0.9967958498626793
Accuracy: 0.9464533957210186



### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'MessageModel.pkl'
pickle.dump(model, open(filename, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.